# 1η Εργαστηριακή Άσκηση : Μέρος Πρώτο , Ορθογράφος

## Επεξεργασία φωνής και φυσικής γλώσσας

### Βλάχος Ιωάννης, 03115013
### ΣΗΜΜΥ, 9ο εξάμηνο, ροή Σ
### Ακαδημαϊκό Έτος 2019-20

### Βήμα 1 : κατασκευή corpus
α) Αρχικά παίρνω τα 2 προτεινόμενα βιβλία και τα κάνω ένα αρχείο merged.text.

In [4]:
p1 = "1661-0.txt"
p2 = "36-0.txt"
filenames = [p1, p2]
with open('merged.txt', 'w', encoding="utf8") as outfile:
    for fname in filenames:
        with open(fname, encoding="utf8") as infile:
            outfile.write(infile.read())
p = "merged.txt"

β) Εκτός από τον μεγαλύτερο όγκο δεδομένων, υπάρχει και μεγαλύτερη ποικιλία στη θεματολογία των λέξεων που χρησιμοποιούνται. Για παράδειγμα, δε θέλω να εμφανίζεται πολλές φορές η χρήση της λέξης "φόνος" αν έχω επιλέξει μόνο ένα αστυνομικό μυθιστόρημα. Εκτός από αυτό, υπάρχει και ποικιλία στον λόγο γενικότερα αν επιλεχθούν περισσότερα από ένα βιβλία. Συμπεριλαμβάνονται έτσι και επίσημοι και πιο καθημερινοί διάλογοι, ενώ οι εκφράσεις και ο τρόπος γραφής 2 διαφορετικών συγγραφέων προσδίδει καλύτερη δειγματοληψία φυσικής γλώσσας.

### Βήμα 2 : προπαρασκευή corpus

In [5]:
def identity_process(s):
    return s


def readFile(pathname, preprocess=identity_process):
    f = open(pathname, "r")
    mylist = []
    lines = f.readlines()
    for line in lines:
        mylist.extend(preprocess(line))
    return mylist

        
def tokenize(s):
    s = s.strip()
    s = s.lower()
    news=""
    for a in s:
        if a==' ' or a=='\n': news+=' '
        elif a.isalpha(): news+=a
    return news.split()

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nbuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
text = "Let's test what it does to @!Symbols and 23numbers! ??:as well,,,a/. as otheRS"

a = nltk.word_tokenize(text)
print(a)
print(tokenize(text))


['Let', "'s", 'test', 'what', 'it', 'does', 'to', '@', '!', 'Symbols', 'and', '23numbers', '!', '?', '?', ':', 'as', 'well', ',', ',', ',', 'a/', '.', 'as', 'otheRS']
['lets', 'test', 'what', 'it', 'does', 'to', 'symbols', 'and', 'numbers', 'as', 'wella', 'as', 'others']


Τα σημεία στίξης τα τυπώνει ξεχωριστά εκτός από την απόστροφο που την κολλάει στις γειτονικές λέξεις. Διαφορετικά κάνει split ανάλογα με τα κενά. Ο ζητούμενος tokenizer που υλοποιήθηκε παραπάνω κρατάει μόνο τους χαρακτήρες και δίνει ως αποτέλεσμα λέξεις μικρών λατινικών χαρακτήρων.

### Βήμα 3 : Κατασκευή λεξικού και αλφαβήτου

Με τη βοήθεια της μεθόδου που φτιάχτηκε παραπάνω διαβάζουμε το κείμενο με παράμετρο την tokenize και μετατρέποντας το set σε λίστα, κρατάμε όλες τις διαφορετικές λέξεις που υπάρχουν στο corpus

In [7]:
tok = readFile(p,tokenize)
tokens = list(set(tok))

In [8]:
tok[300:320]

['observerexcellent',
 'for',
 'drawing',
 'the',
 'veil',
 'from',
 'mens',
 'motives',
 'and',
 'actions',
 'but',
 'for',
 'the',
 'trained',
 'reasoner',
 'to',
 'admit',
 'such',
 'intrusions',
 'into']

Χρησιμοποιώντας το λεξικό λέξεων, παίρνουμε όλα τα γράμματα που υπάρχουν στις λέξεις του λεξικού. 
Φτιάχνοντας ομοίως ένα set από αυτά κρατάμε όλους τους διαφορετικούς χαρακτήρες του corpus

In [9]:
l = [list(i) for i in tokens]
flat_list = [item for sublist in l for item in sublist]
symbols=list(set(flat_list))
print(symbols)

['p', 'k', 'q', 'ç', 'h', 's', 'w', 'j', 'f', 't', 'u', 'n', 'g', 'i', 'â', 'z', 'v', 'æ', 'l', 'è', 'a', 'é', 'r', 'o', 'œ', 'm', 'c', 'd', 'b', 'y', 'à', 'e', 'x']


### Βήμα 4

Αντιστοιχίζουμε κάθε χαρακτήρα σε ένα αύξοντα index. Κάνουμε το ίδιο και για τις λέξεις σε περίπτωση που χρειαστεί να βγάλουμε output μια ολόκληρη λέξη.

In [8]:
def makeOFST(alphabet,s):
    f= open(s,"w+")
    f.write("<epsilon> 0\n")
    count=1
    for i in alphabet:
        if i=='\n': f.write("<newline>" + '\t' + str(count) + '\n')
        elif i==' ': f.write("<space>" + '\t' + str(count) + '\n')
        else: f.write(i + '\t' + str(count) + '\n')
        count+=1
    f.close()
    return f

In [9]:
makeOFST(symbols, "chars.syms")
makeOFST(tokens, "out.syms")

<_io.TextIOWrapper name='out.syms' mode='w+' encoding='UTF-8'>

### Βήμα 5

In [10]:
f = open('text.fst', mode = 'w')

def format_arc(src, dst, src_sym, dst_sym, w):
    f.write(str(src)+" "+str(dst)+" "+src_sym+" "+dst_sym+" "+str(w)+'\n')
    return

for i in range(len(symbols)):
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[i], w=0)
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym="<epsilon>", w=1)
    format_arc(
        src=0, dst=0, src_sym="<epsilon>", dst_sym=symbols[i], w=1)
    for j in range(len(symbols)):
        if j!=i:
            format_arc(
                src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[j], w=1)
f.write('0')


f.close()

Βάζουμε τη μοναδική κατάσταση ως τελική για να μπορούμε να συνδυάσουμε το αυτόματο με άλλα fst.

α) Έχοντας φτιάξει το αρχείο text.fst στο οποίο υπάρχουν όλες οι ζητούμενες μεταβάσεις του αυτόματου, τρέχουμε την εντολή 

fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols text.fst transducer.fst

για να φτιάξουμε τον transducer. 
Αν σε αυτόν τον transducer εφαρμόσουμε τον αλγόριθμο ShortestPath για μία συγκεκριμένη είσοδο θα μας δώσει την ίδια τη λέξη μαζί με το πλήθος των μεταβάσεων σε διαφορετικά γράμματα. Δηλαδή αν δεν είχε δύο ίδια συνεχόμενα γράμματα θα μας έδινε το μήκος της λέξης -1.

β) Ωστόσο η μέθοδος αυτή δε μας δίνει κάποια προτίμηση ως προς το ποια γράμματα αλλάζουμε και με ποια πράξη.
Για παράδειγμα το deletion ενός χαρακτήρα μπορεί να κοστίζει περισσότερο αν αυτός ο χαρακτήρας είναι σπάνιος, μια και χαρακτηρίζει περισσότερο τη λέξη εισόδου από ό,τι ένα φωνήεν. Το ίδιο ισχύει και για το insertion.
Επομένως σε αυτά τα βάρη θα μπορούσαμε να βάλουμε τιμές αντιστρόφως ανάλογες στη συχνότητα εμφάνισης των γραμμάτων που εισέρχονται ή διαγράφονται.

Επίσης, η μετάβαση από ένα γράμμα σε ένα άλλο θα πρέπει επίσης να κοστίζει παραπάνω αν το ζεύγος που προκύπτει είναι σπάνιο, καθώς διαφοροποιεί σημαντικά τη λέξη εισόδου. Έτσι, αν είχαμε στατιστικά για τη συχνότητα εμφάνισης ζεύγων γραμμάτων θα μπορούσαμε πάλι να εφαρμόσουμε κανονικοποιημένα βάρη αντιστρόφως ανάλογα στην συχνότητα με την οποία προκύπτουν τα ζεύγη. 

### Βήμα 6

In [11]:
g = open('acceptor.fst', mode = 'w')
count=1
for word in tokens:
    prev = "<epsilon>"
    for i in range(len(word)):
        l = word[i]
        if i==len(word)-1: s=word
        else: s="<epsilon>"
        if prev!="<epsilon>":
            g.write(str(count-1)+" "+str(count)+" "+l+" "+l +" "+'0''\n')
            count+=1
        else:
            g.write('0'+" "+str(count)+" "+l+" "+l+" "+'0'+'\n')
            count+=1
        prev=l
    g.write(str(count-1)+'\n')
g.close()

α) Με τον ίδιο τρόπο δημιουργούμε με την εντολή
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols acceptor.fst accept.fst

τον εκτελέσιμο αποδοχέα.

Ουσιαστικά για κάθε λέξη του λεξικού tokens εισάγουμε ένα μονοπάτι που αποδέχεται αυτή τη λέξη.

β) Εφαρμόζουμε στη συνέχεια κατά σειρά τις ζητούμενες πράξεις πάνω στο accept.fst

fstrmepsilon accept.fst acceptA.fst

fstdeterminize acceptA.fst acceptB.fst

fstminimize acceptB.fst acceptC.fst

Η πρώτη από αυτές μετατρέπει το αυτόματο σε transducer χωρίς ε-καταστάσεις. Ο fstdeterminize μετατρέπει την είσοδο σε ντετερμινιστικό αυτόματο, ενώ η τελευταία εντολή ελαχιστοποιεί το αυτόματο μικραίνοντας κατά το δυνατό τον αριθμό των καταστάσεων.

### Βήμα 7
α) Δημιουργούμε τον ορθογράφο με την fstcompose

fstcompose transducer.fst acceptC.fst min_dist_check.fst

Δοκιμάζουμε επίσης να βάλουμε διαφορετικά βάρη για το αν τα γράμματα είναι σπάνια. Έτσι ώστε αν διαγράψουμε ή εισάγουμε ένα γράμμα που αλλάζει πολύ την λέξη να μην προτιμηθεί από το αυτόματο. Το ίδιο και αν αντικαταστήσουμε ένα σπάνιο γράμμα ή βάλουμε ένα σπάνιο γράμμα στη θέση ενός φωνήεντος που μπορεί να αλλάζει πολύ τη λέξη.

In [47]:
f = open('weights.fst', mode = 'w')
weights = [30, 100, 100, 100, 40, 20, 35, 10, 15, 19, 32, 5, 25, 100, 60, 21, 23, 10, 7, 8, 14, 100, 19, 15, 13, 8,
           24, 26, 9, 10, 100, 4, 100]


for i in range(len(symbols)):
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[i], w=0)
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym="<epsilon>", w=weights[i])
    format_arc(
        src=0, dst=0, src_sym="<epsilon>", dst_sym=symbols[i], w=weights[i])
    for j in range(len(symbols)):
        if j!=i:
            format_arc(
                src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[j], w=weights[i]+weights[j])
f.write('0')


f.close()
from IPython.display import Image

Θα σχολιάσουμε το ερώτημα για τη συμπεριφορά του weighted αυτόματου στο ερώτημα 8 που θα δούμε αναλυτικότερα τη συμποεριφορά του non-weighted πάνω σε δείγμα λέξεων. Φτιάχνουμε ωστόσο το fst με τις παρακάτω εντολές:

fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols weigts.fst third.fst

fstcompose third.fst acceptC.fst weighted_edits.fst

β) Φτιάχνουμε το αρχείο εισόδου που είναι απλά η λέξη που θέλουμε να δώσουμε ως είσοδο σε μορφή fst.

fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait input1.fst | dot -Tjpg > cit.jpg

![title](cit.jpg)

In [13]:
letters = list('cit')
def inp(letters,file):
    s = 0

    a = open(file, 'w')

    for i in range(0, len(letters)):
        a.write(str(s)+" "+str(s+1)+" "+letters[i]+" "+letters[i]+'\n')
        s += 1
    a.write(str(s))
    a.close()
    
inp(letters, 'input.fst')

Στη συνέχεια κάνουμε compile το αρχείο εξόδου fst, κάνουμε fstcompose με τον ορθογράφο και αφού καλέσουμε την shortestPath για ν=5 βγάζουμε το αποτέλεσμα σε εικόνα για να είναι φανερό ποιες λέξεις είναι κοντινότερες και ποιές πράξεις εφαρμόστηκαν.

fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols input.fst input1.fst

fstcompose input1.fst min_dist_check.fst fourth.fst

fstshortestpath --nshortest=5 fourth.fst out.fst

fstrmepsilon out.fst out.fst

fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait out.fst | dot -Tjpg > closest.jpg

![title](closest.jpg)

Από την εικόνα βλέπουμε ότι η λέξη 'cit' σύμφωνα με τον ορθογράφο είναι πιο κοντά και συγκεκριμένα απέχει από τις 5 λέξεις απόσταση 1 σύμφωνα με την απόσταση Levenshtein. Οι λέξεις είναι οι παρακάτω:

hit

sit 

city

cut

bit

### Βήμα 8

Αφού κατεβάσουμε το αρχείο επιλέγουμε τυχαία 20 από τις λανθασμένες λέξεις και τις βάζουμε σε μία λίστα.

In [14]:
words = ['vistors', 'magnifiscant', 'oppisite', 'cirtain', 'neccasary', 'acommodation', 'chalenges', 'lagh', 'perple', 'offten',
        'somone', 'possition', 'sissors', 'speaical', 'experances', 'receit', 'compair', 'arrainged', 'poartry', 'arnt']

correct = ['visitors', 'magnificent', 'opposite', 'certain', 'necessary', 'accommodation', 'challenges', 'laugh', 'purple',
          'often', 'someone', 'position', 'scissors', 'special', 'experiences', 'receipt', 'compare', 'arranged', 'poetry', 'aunt']

print(len(words))
print(len(correct))

20
20


Για κάθε μία από αυτές δημιουργούμε το αντίστοιχο αρχείο εισόδου όπως στην περίπτωση της λέξης 'cit'

In [15]:
files = []
s = 'input'
for i in range(len(words)):
    files.append(s+str(i)+'.fst')
    inp(words[i], files[i])

Για κάθε ένα από τα παραπάνω αρχεία εκτελώ το παρακάτω script το οποίο βάζει στο αρχείο a.txt την κοντινότερη προς την είσοδο λέξη που αποδέχεται ο ορθογράφος.



In [16]:
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols $1 inputA.fst
fstcompose inputA.fst min_dist_check.fst fourth.fst
fstshortestpath --nshortest=1 fourth.fst out.fst
fstrmepsilon out.fst out.fst
fsttopsort out.fst out.fst
fstprint out.fst word.txt
cut -f4 word.txt > res.txt
head -n -1 res.txt >> a.txt
echo 'new_word' >> a.txt
sed -i -e 's/<epsilon>//g' a.txt

SyntaxError: invalid syntax (<ipython-input-16-b04c05bfdbd1>, line 1)

Επεξεργάζομαι στη συνέχεια το a.txt και βγάζω τις προβλέψεις

In [17]:
pred = []
f = open("a.txt", 'r')
lines = f.readlines()
word = ""
for l in lines:
    if l!='new_word\n':
        word+=l.rstrip()
    else:
        pred.append(word)
        word=""

import pandas as pd
d = pd.DataFrame({'word':words, 'correct word':correct, 'prediction':pred})
print(d)

            word   correct word   prediction
0        vistors       visitors     visitors
1   magnifiscant    magnificent  magnificent
2       oppisite       opposite     opposite
3        cirtain        certain      certain
4      neccasary      necessary    necessary
5   acommodation  accommodation  combination
6      chalenges     challenges      changes
7           lagh          laugh        laugh
8         perple         purple       purple
9         offten          often        often
10        somone        someone      someone
11     possition       position     position
12       sissors       scissors     scissors
13      speaical        special      special
14    experances    experiences  experiences
15        receit        receipt      receipt
16       compair        compare     complain
17     arrainged       arranged     arranged
18       poartry         poetry       portly
19          arnt           aunt          art


Βλέπουμε ότι υπάρχουν αρκετές λάθος προβλέψεις, γεγονός που οφείλεται στα ορθογραφικά λάθη των λέξεων.
Συγκεκριμένα, ο ορθογράφος εκτός από κάποιες λέξεις που δεν υπάρχουν στο λεξικό, επιλέγει σίγουρα λέξη με την ίδια ελάχιστη απόσταση από τη σωστή λέξη. Οι προβλέψεις αυτές έχουν ελάχιστη απόσταση από τη λέξη αλλά είναι αρκετά πιο "ξένες" τις σωστές γιατί αλλάζουν κάποια χαρακτηριστικά σύμφωνα των λέξεων, όπως στην περίπτωση του complain.

Ωστόσο έχουν γίνει αρκετές σωστές προβλέψεις. Τέλος το λάθος είναι δικαιολογημένο στη λέξη "arnt" που είναι και μικρή λέξη και περιέχει το πιο ασυσχέτιστο προς τη σωστή λέξη "r". 

Ασχολόυμαστε στη συνέχεια με τον weigthed ορθογράφο τον οποίο υλοποιούμε ομοίως με τον απλό:

In [18]:
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols $1 inputA.fst
fstcompose inputA.fst weighted_edits.fst fourth.fst
fstshortestpath --nshortest=1 fourth.fst out.fst
fstrmepsilon out.fst out.fst
fsttopsort out.fst out.fst
fstprint out.fst word.txt
cut -f4 word.txt > res.txt
head -n -1 res.txt >> w.txt
echo 'new_word' >> w.txt
sed -i -e 's/<epsilon>//g' w.txt

SyntaxError: invalid syntax (<ipython-input-18-688f018f71f8>, line 1)

In [19]:
pred = []
f = open("w.txt", 'r')
lines = f.readlines()
word = ""
for l in lines:
    if l!='new_word\n':
        word+=l.rstrip()
    else:
        pred.append(word)
        word=""

import pandas as pd
d = pd.DataFrame({'word':words, 'correct word':correct, 'prediction':pred})
print(d)

            word   correct word   prediction
0        vistors       visitors     visitors
1   magnifiscant    magnificent  magnificent
2       oppisite       opposite     opposite
3        cirtain        certain      certain
4      neccasary      necessary    necessary
5   acommodation  accommodation       common
6      chalenges     challenges      changes
7           lagh          laugh        laugh
8         perple         purple       purple
9         offten          often        often
10        somone        someone      someone
11     possition       position     position
12       sissors       scissors     scissors
13      speaical        special      special
14    experances    experiences  experiences
15        receit        receipt      receipt
16       compair        compare      compare
17     arrainged       arranged     arranged
18       poartry         poetry       poetry
19          arnt           aunt        arent


Παρατηρούμε ότι συμπεριφέρεται παρόμοια με τον απλό ορθογράφο και ότι μάλλον δεν υπάρχει η αναγκαιότητα μεγάλων βαρών στον transducer. Υπενθυμίζουμε ότι κάποιες λέξεις είναι αρκετά ξένες ως προς τη σωστή γιατί δεν υπάρχουν στο αρχικό λεξικό. Ο σκοπός των διαφορετιών βαρών επιτυγχάνεται στο τελευταίο παράδειγμα όπου το arnt φέρνει πιο πολύ στη λέξη arent μια και απλά λείπει ένα "e". Βέβαια σύμφωνα με τα δεδομένα εισόδου η σωστή λέξη είναι η aunt.

### Βήμα 9
α) Ξαναγράφουμε την tokenize ώστε να επιστρέφει πρόταση. H readFile επιστρέφει κάθε γραμμή ως μία πρόταση.

In [6]:
def tokenize(s):
    s = s.strip()
    s = s.lower()
    news=""
    for a in s:
        if a==' ' or a=='\n': news+=' '
        elif a.isalpha(): news+=a
    return news.split()

a=tokenize("My -name 9is Bo23nd, .././ JAMEs... @#Bond")
print(a)

['my', 'name', 'is', 'bond', 'james', 'bond']


In [8]:
def readFile(pathname, preprocess=identity_process):
    f = open(pathname, "r",encoding="utf8")
    mylist = []
    lines = f.readlines()
    for line in lines:
        mylist.append(preprocess(line))
    return mylist

senten = readFile(p, tokenize)
print(senten[100:110])

[[], ['one', 'nightit', 'was', 'on', 'the', 'twentieth', 'of', 'march', 'i', 'was', 'returning', 'from', 'a'], ['journey', 'to', 'a', 'patient', 'for', 'i', 'had', 'now', 'returned', 'to', 'civil', 'practice', 'when'], ['my', 'way', 'led', 'me', 'through', 'baker', 'street', 'as', 'i', 'passed', 'the', 'wellremembered'], ['door', 'which', 'must', 'always', 'be', 'associated', 'in', 'my', 'mind', 'with', 'my', 'wooing', 'and'], ['with', 'the', 'dark', 'incidents', 'of', 'the', 'study', 'in', 'scarlet', 'i', 'was', 'seized', 'with', 'a'], ['keen', 'desire', 'to', 'see', 'holmes', 'again', 'and', 'to', 'know', 'how', 'he', 'was', 'employing', 'his'], ['extraordinary', 'powers', 'his', 'rooms', 'were', 'brilliantly', 'lit', 'and', 'even', 'as', 'i'], ['looked', 'up', 'i', 'saw', 'his', 'tall', 'spare', 'figure', 'pass', 'twice', 'in', 'a', 'dark', 'silhouette'], ['against', 'the', 'blind', 'he', 'was', 'pacing', 'the', 'room', 'swiftly', 'eagerly', 'with', 'his']]


In [10]:
from gensim.models import Word2Vec
model = Word2Vec(senten, size=100, window=5, workers=4)

In [11]:
model.train(senten, total_examples=len(senten), epochs=1000)

(106777342, 170524000)

In [12]:
# get ordered vocabulary list
voc = model.wv.index2word

In [26]:
dim = model.vector_size

# get most similar words
sim = model.wv.most_similar('holmes')

# Convert to numpy 2d array (n_vocab x vector_size)
def to_embeddings_Matrix(model):  
    embedding_matrix = np.zeros((len(model.wv.vocab), model.vector_size))
    word2idx = {}
    for i in range(len(model.wv.vocab)):
        embedding_matrix[i] = model.wv[model.wv.index2word[i]]
        word2idx[model.wv.index2word[i]] = i
    return embedding_matrix, model.wv.index2word, word2idx

In [94]:
import random
test = ['clue']
for i in range(10):
    test.append(voc[random.choice(range(len(voc)))])

for i in test:
    print(i)
    print(model.wv.most_similar(i))
    print()

clue
[('clues', 0.6851588487625122), ('inkling', 0.6116672158241272), ('slightest_clue', 0.5809398889541626), ('know', 0.5430039763450623), ('foggiest_idea', 0.5362423658370972), ('hint', 0.5173760652542114), ('indication', 0.5164186954498291), ('faintest_idea', 0.511157751083374), ('wonder', 0.5002449154853821), ('hints', 0.492409884929657)]

abandoned


C:\anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('abandoning', 0.6218770146369934), ('Abandoned', 0.580659031867981), ('abandonded', 0.5756361484527588), ('abandon', 0.552590548992157), ('derelict', 0.5503709316253662), ('abandonned', 0.5397235155105591), ('deserted', 0.5132448673248291), ('neglected', 0.5129300355911255), ('scrapped', 0.49730584025382996), ('discarded', 0.4953513443470001)]

beside
[('alongside', 0.5863358378410339), ('beneath', 0.5278612375259399), ('front', 0.5269509553909302), ('Beside', 0.5262925624847412), ('crouched_beside', 0.5186084508895874), ('astride', 0.5054730176925659), ('atop', 0.5051977038383484), ('underneath', 0.49534720182418823), ('overlooking', 0.49429577589035034), ('perched', 0.49401241540908813)]

grasped
[('grasp', 0.7057170867919922), ('grasping', 0.657967746257782), ('grasps', 0.6562371850013733), ('comprehended', 0.5534980297088623), ('embraced', 0.5434234142303467), ('understood', 0.5398856401443481), ('comprehend', 0.5192852020263672), ('realized', 0.4915735423564911), ('cottoned', 0.

Έχοντας φτιάξει το vocabulary του corpus επιλέγουμε εντελώς τυχαία 10 λέξεις από αυτό και τυπώνουμε τις πιο συσχετισμένες προς αυτές λέξεις.

Παρατηρούμε ότι δεν είναι αρκετά επιτυχημένη η συσχέτιση των παρόμοιων λέξεων, εκτός ίσως αυτών που έχουν μεγαλύτερα ποσοστά συσχέτισης.
Για παράδειγμα η λέξη clue (στοιχείο) είναι σημασιολογικά κοντά στη λέξη trace (ίχνος), αφού μιλάμε για υποθέσεις και σκηνές εγκλήματος στην περίπτωση του βιβλίου του Sherlock Holmes. Βέβαια οι υπόλοιπες λέξεις που φαίνονται ως αποτέλεσμα όπως το 'animal' δεν έχουν κάποια εμφανή σχέση με τη λέξη clue.

Σε ό,τι αφορά τα υπόλοιπα αποτελέσματα δεν φαίνεται να έχει γίνει καλό match αν και οι λέξεις είναι πιο αφηρημένες (επίθετα, επιρρήματα, ...). Ασφαλώς βέβαια το μικρό window πιθανώς να μη βοηθάει στην καλύτερη εκπαίδευση του συστήματος. Ενδεχομένως βέβαια και ο όγκος των δεδομένων να μην είναι αρκετά μεγάλος ώστε να μας δώσει καλά αποτελέσματα.

Αυξάνουμε αρχικά το window ώστε να συμπεριλάβουμε περισσότερες λέξεις στη σφαίρα συσχέτισης των λέξεων. Αυτό ενδεχομένως να βοηθήσει αν σκεφτούμε ότι μια λέξη στενά συνδεδεμένη με μια άλλη μπορεί να μην απέχει μόνο απόσταση 2 αλλά παραπάνω.

In [ ]:
model = Word2Vec(senten, size=100, window=10, workers=4)
model.train(senten, total_examples=len(senten), epochs=1000)

In [ ]:
for i in test:
    print(i)
    print(model.wv.most_similar(i))
    print()

Παρατηρούμε μια ελαφριά βελτίωση στις επιλεγμένες λέξεις. Για παράδειγμα στη λέξη "εγκυκλοπέδια" υπάρχει η λέξη "παράγραφος" που δεν υπήρχε πριν. Επίσης στη λέξη desk για παράδειγμα, υπάρχουν οι λέξεις table, opposite, triangular που ταιριάζουν σε ένα γραφείο και δεν υπήρχαν πριν. Συνοψίζοντας, παρατηρούμε ότι υπάρχει κάποια βελτίωση μετά την αύξηση του window_size. 

Αυξάνουμε τώρα τον αριθμό των epochs, ο οποίος ήταν ήδη 1000. Τον αυξάνουμε κατά μια τάξη μεγέθους και βλέπουμε αν αυτή η παράμετρος συμβάλλει σημαντικά στην εκπαίδευση του μοντέλου.

In [ ]:
model = Word2Vec(senten, size=100, window=10, workers=4)
model.train(senten, total_examples=len(senten), epochs=5000)

for i in test:
    print(i)
    print(model.wv.most_similar(i))
    print()

Όπως είχαμε προβλέψει η αύξηση στον αριθμό των εποχών δε βοήθησε πολύ στη εύρεση πιο συσχετισμένων αποτελεσμάτων. Συγκεγκριμένα, έχουμε πάνω κάτω ίδια αποτελέσματα με πριν, ενώ έχουμε χάσει κάποια επιτυχημένα αποτελέσματα όπως για παράδειγμα το table του desk ή το "powerful knock" και το "strength" του convinced. 

Γενικά, συμπεραίνουμε ότι είναι καλή στρατηγική να χρησιμοποιούμε μεγαλύτερο window ώστε να αυξάνουμε την ακτίνα επιρροής και συμπεριλαμβάνουμε περισσότερα δεδομένα. Αντίθετα, ο μεγάλος αριθμός epochs δρα αρνητικά στην εκπαίδευση του μοντέλου έχοντας ως αποτέλεσμα ένα είδος overfitting που δίνει ασυσχέτιστα αποτελέσματα.

# Μέρος 1ο : Ορθογράφος

### Βήμα 10

Με τη βοήθεια της βιβλιοθήκης collection βρίσκουμε τη συχνότητα των λέξεων στο αρχικό σύνολο λέξεων.
Διαιρούμε ύστερα με το πλήθος των λέξεων του corpus για να βρούμε τις πιθανότητες εμφάνισης. Εφαρμόζουμε το ίδιο και για τα γράμματα τα οποία είναι όλα αποθκευμένα στη λίστα flat_list

In [22]:
import collections
counter = collections.Counter(tok)

In [23]:
word_counter = dict(counter)

letter_counter = dict(collections.Counter(flat_list))
letter_counter
for i in letter_counter:
    letter_counter[i] /= len(flat_list)

for i in word_counter:
    word_counter[i] /= len(tok)

In [24]:
letter_counter

{'a': 0.07197618733799042,
 'b': 0.017683757304160625,
 'u': 0.03280831246430298,
 'n': 0.0725253723474364,
 'd': 0.04798778612538992,
 'c': 0.039541320680110716,
 'e': 0.12455516014234876,
 'p': 0.02794253328061157,
 'i': 0.07733623303018321,
 'f': 0.015882430473177805,
 'l': 0.05316110891437107,
 'r': 0.07273406265102587,
 'h': 0.026964983963797724,
 's': 0.0764904881156364,
 't': 0.06902157198717104,
 'o': 0.05887263301260929,
 'm': 0.025383331136593294,
 'v': 0.011159439391942357,
 'y': 0.017310311497737357,
 'k': 0.009358112560959536,
 'g': 0.030556653925574447,
 'w': 0.01321339132727033,
 'x': 0.002921664250252625,
 'j': 0.0017793594306049821,
 'z': 0.001109353719080884,
 'q': 0.0015157506260709108,
 'é': 0.00010983700188919643,
 'æ': 3.295110056675893e-05,
 'œ': 2.1967400377839286e-05,
 'à': 1.0983700188919643e-05,
 'ç': 1.0983700188919643e-05,
 'è': 1.0983700188919643e-05,
 'â': 1.0983700188919643e-05}

### Βήμα 11

α) Υπολογίζουμε αρχικά τη ζητούμενη μέση τιμή.

In [42]:
from statistics import mean
word_mean = mean(word_counter[k] for k in word_counter)
import math
word_mean = -math.log(word_mean,10)
word_mean

4.086502198970369

β) Φτιάχνουμε τον μετατροπέα όπως ακριβώς και στο βήμα 5 του προπαρασκευαστικού εργαστηρίου.

Είναι πολύ σημαντικό να εφαρμόσουμε τον ορισμό του βάρους βάζοντας ως βάρος τον αντίθετο του log της τιμής που βρήκαμε παραπάνω. Έτσι το βάρος w θα είναι της ίδιας τάξης με τα βάρη του acceptor. Σε διαφορετική περίπτωση οι υψηλές τιμές βαρών του acceptor θα υπερίσχυαν στο συνολικό σύστημα επιλέγοντας έτσι απλά την πιο πολυσύχναστη λέξη ("the") και αγνοώντας τον transducer Levenshtein.

Σημειώνουμε ότι τη μέση τιμή θα μπορούσαμε να την έχουμε βρει απλά ως 1/len(tokens) αφού οι πιθανότητες αθροίζουν στο 1.

In [44]:
f = open('word_unigram.fst', mode = 'w')

def format_arc(src, dst, src_sym, dst_sym, w):
    f.write(str(src)+" "+str(dst)+" "+src_sym+" "+dst_sym+" "+str(w)+'\n')
    return

for i in range(len(symbols)):
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[i], w=0)
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym="<epsilon>", w=word_mean)
    format_arc(
        src=0, dst=0, src_sym="<epsilon>", dst_sym=symbols[i], w=word_mean)
    for j in range(len(symbols)):
        if j!=i:
            format_arc(
                src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[j], w=word_mean)
f.write('0')
f.close()

γ) Επαναλαμβάνουμε το ίδιο για το unigram μοντέλο.

In [45]:
letter_mean = mean(letter_counter[k] for k in letter_counter)
letter_mean = -math.log(letter_mean,10)
f = open('letter_unigram.fst', mode = 'w')

def format_arc(src, dst, src_sym, dst_sym, w):
    f.write(str(src)+" "+str(dst)+" "+src_sym+" "+dst_sym+" "+str(w)+'\n')
    return

for i in range(len(symbols)):
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[i], w=0)
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym="<epsilon>", w=letter_mean)
    format_arc(
        src=0, dst=0, src_sym="<epsilon>", dst_sym=symbols[i], w=letter_mean)
    for j in range(len(symbols)):
        if j!=i:
            format_arc(
                src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[j], w=letter_mean)
f.write('0')
f.close()
letter_mean

1.5185139398778873

Κάνουμε compile και τα 2 αρχεία με τις εντολές 

In [30]:
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols word_unigram.fst word.fst
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols letter_unigram.fst letter.fst

SyntaxError: invalid syntax (<ipython-input-30-528e4e0b320b>, line 1)

Η αφέλεια της παραπάνω μεθόδου οφείλεται στο γεγονός ότι έχουμε το ίδιο βάρος για κάθε πράξη που εφαρμόζουμε στη λέξη εισαγωγής. 
Όπως έχουμε αναφέρει και σε προηγούμενο ερώτημα, θα ήταν προτιμότερο να εξειδικεύσουμε τις τιμές για κάθε ζεύγος γραμμάτων ξεχωριστά ανάλογα με τη συχνότητα με την οποία συμβαίνουν αυτά τα λάθη. Έτσι θα μπορούσαμε να αντικαταστήσουμε την παραπάνω τιμή με την πιθανότητα να αντικατασταθεί ένα γράμμα με ένα άλλο κτλπ.

Η απλότητα βέβαια της απόστασης Levenshtein, που μας δίνει τον ελάχιστο αριθμό πράξεων επί τον συντελεστή που δώσαμε είναι αρκετό για να κάνουμε string matching.

### Βήμα 12
α) Ακολουθώντας τις οδηγίες βάζουμε το βάρος της λέξης στην πρώτη μετάβαση προς το μονοπάτι του fst που ορίζει την λέξη.

In [28]:
g = open('log_acceptor.fst', mode = 'w')
count=1
for word in tokens:
    prev = "<epsilon>"
    val = -math.log(word_counter[word],10)
    for i in range(len(word)):
        l = word[i]
        if prev!="<epsilon>":
            g.write(str(count-1)+" "+str(count)+" "+l+" "+l +" "+'0'+'\n')
            count+=1
        else:
            g.write('0'+" "+str(count)+" "+l+" "+l+" "+str(val)+'\n')
            count+=1
        prev=l
    g.write(str(count-1)+'\n')
g.close()

β) Ομοίως με το ερώτημα 6 έχουμε τα εξής.

In [31]:
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols log_acceptor.fst word_acceptor.fst
fstrmepsilon word_acceptor.fst word_acceptor.fst
fstdeterminize word_acceptor.fst word_acceptor.fst
fstminimize word_acceptor.fst word_acceptor.fst

SyntaxError: invalid syntax (<ipython-input-31-f6a7af0773d9>, line 1)

γ) Κάνουμε το ίδιο και για τα γράμματα, μόνο που σε κάθε μετάβαση βάζουμε τον λογάριθμο της πιθανότητας του αντίστοιχου γράμματος με το οποίο γίνεται η μετάβαση στον acceptor.

In [29]:
g = open('let_log_acceptor.fst', mode = 'w')
count=1
for word in tokens:
    prev = "<epsilon>"
    for i in range(len(word)):
        l = word[i]
        val = -math.log(letter_counter[l],10)
        if prev!="<epsilon>":
            g.write(str(count-1)+" "+str(count)+" "+l+" "+l +" "+str(val)+'\n')
            count+=1
        else:
            g.write('0'+" "+str(count)+" "+l+" "+l+" "+str(val)+'\n')
            count+=1
        prev=l
    g.write(str(count-1)+'\n')
g.close()

In [32]:
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols let_log_acceptor.fst let_acceptor.fst
fstrmepsilon let_acceptor.fst let_acceptor.fst
fstdeterminize let_acceptor.fst let_acceptor.fst
fstminimize let_acceptor.fst let_acceptor.fst

SyntaxError: invalid syntax (<ipython-input-32-9db8bd79d8b4>, line 1)

### Βήμα 13
α) Με την fstcompose δημιουργούμε τα αντίστοιχα binary files

In [33]:
fstarcsort word.fst word.fst
fstarcsort word_acceptor.fst word_acceptor.fst
fstcompose word.fst word_acceptor.fst word_checker.fst

SyntaxError: invalid syntax (<ipython-input-33-bb070daa894a>, line 1)

β) Επαναλαμβάνουμε για το unigram μοντέλο:

In [34]:
fstarcsort let_acceptor.fst let_acceptor.fst
fstcompose word.fst let_acceptor.fst let_checker.fst

SyntaxError: invalid syntax (<ipython-input-34-bcad5a021f1e>, line 1)

γ) Αναλύουμε στη συνέχεια τη συμπεριφορά των 2 αποτελεσμάτων με είσοδο τη λέξη "cit"

In [35]:
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols input.fst word_input.fst
fstcompose word_input.fst word_checker.fst out.fst
fstshortestpath --nshortest=3 out.fst out.fst
fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait out.fst | dot -Tjpg > cit_log.jpg

SyntaxError: invalid syntax (<ipython-input-35-f259e351defd>, line 1)

In [36]:
fstcompose word_input.fst let_checker.fst out1.fst
fstshortestpath --nshortest=3 out1.fst out1.fst
fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait out1.fst | dot -Tjpg > cit_unigram.jpg

SyntaxError: invalid syntax (<ipython-input-36-a384cd7f5404>, line 1)

Τα 3 shortest paths για τον word model speller φαίνονται στο παρακάτω διάγραμμα.
![title](cit_log.jpg)

Αντίστοιχα για τον unigram spell checker το αποτέλεσμα είναι το εξής:

![title](cit_unigram.jpg)



Ουσιαστικά για τον 1ο speller οι κοντινότερες λέξεις είναι οι "it", "pit", "cut" ενώ για τον δεύτερο οι λέξεις "it", "sit", "lit".
Αυτή η συμπεριφορά είναι και ενδεικτική του τρόπου λειτουργίας των 2 μηχανών, μια και η πρώτη καθορίζεται ως ένα βαθμό από τη συχνότητα των λέξεων που ενδεχομένως να αντικαταστήσουν την λέξη-είσοδο, ενώ η δεύτερη καθορίζεται περισσότερο από τα συχνά γράμματα που μπορεί να διορθώσουν ένα λάθος. 

Έτσι για παράδειγμα στην πρώτη περίπτωση οι λέξεις pit, cut είναι πιο συχνές από τις sit, lit, ενώ αντίθετα τα γράμματα "l", "s", είναι πιο πολυσύχναστα από τα "p", "u" που χρησιμοποιήθηκαν ως αντικατάσταση στην πρώτη περίπτωση. Αυτό αποτυπώνεται και στα βάρη των παραπάνω fst όπου οι αντικατστάσεις κοστίζουν.

### Βήμα 14

α) Βάζω την είσοδο σε ένα test.txt

In [35]:
f = open("test.txt", "r")
lines = f.readlines()

In [36]:
misspelled = []
truth = []
for l in lines:
    s = l.split(":")
    correct = s[0]
    wr = s[1]
    wrong = wr.split(" ")
    for i in range(1,len(wrong)):
        misspelled.append(wrong[i].strip('\n'))
        truth.append(correct)
for i in range(len(misspelled)):
    print(misspelled[i]+" "+truth[i])

contenpted contented
contende contented
contended contented
contentid contented
begining beginning
problam problem
proble problem
promblem problem
proplen problem
dirven driven
exstacy ecstasy
ecstacy ecstasy
guic juice
juce juice
jucie juice
juise juice
juse juice
localy locally
compair compare
pronounciation pronunciation
transportibility transportability
miniscule minuscule
independant independent
independant independent
aranged arranged
arrainged arranged
poartry poetry
poertry poetry
poetre poetry
poety poetry
powetry poetry
leval level
basicaly basically
triangulaur triangular
unexpcted unexpected
unexpeted unexpected
unexspected unexpected
stanerdizing standardizing
varable variable
futher further
monitering monitoring
biscits biscuits
biscutes biscuits
biscuts biscuits
bisquits biscuits
buiscits biscuits
buiscuts biscuits
avaible available
seperate separate
neccesary necessary
necesary necessary
neccesary necessary
necassary necessary
necassery necessary
neccasary necessary
def

β) Τρέχουμε στη συνέχεια στο τερματικό το εξής script ώστε να μπορούμε ταυτόχρονα να τρέξουμε και bash εντολές.

In [55]:
import os

for i in misspelled:
    inp(i, "input.fst")
    os.system("fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols input.fst inputA.fst")
    os.system("fstcompose inputA.fst word_checker.fst fourth.fst")
    os.system("fstshortestpath --nshortest=1 fourth.fst out.fst")
    os.system("fstrmepsilon out.fst out.fst")
    os.system("fsttopsort out.fst out.fst")
    os.system("fstprint out.fst word.txt")
    os.system("cut -f4 word.txt > res.txt")
    os.system("head -n -1 res.txt >> a.txt")
    os.system("echo 'new_word' >> a.txt")
    os.system("sed -i -e 's/<epsilon>//g' a.txt")


pred = []
f = open("a.txt", 'r')
lines = f.readlines()
word = ""
for l in lines:
    if l!='new_word\n':
        word+=l.rstrip()
    else:
        pred.append(word)
        word=""

print(pred)

['visitors', 'magnificent', 'opposite', 'certain', 'necessary', 'combination', 'changes', 'laugh', 'purple', 'often', 'someone', 'position', 'scissors', 'special', 'experiences', 'receipt', 'complain', 'arranged', 'portly', 'art', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


Ως αποτέλεσμα έχουμε το παρακάτω pred με βάση το οποίο βρίσκουμε το ποσοστό επιτυχίας σε όλες τις λέξεις αλλά και αν κρατήσουμε μόνο τις λέξεις που υπάρχουν στο λεξικό μας.

In [62]:
pred = ['consented', 'continue', 'continued', 'contents', 'beginning', 'problem', 'problem', 'problem', 'people', 'given', 'eustace', 'eustace', 'guns', 'june', 'judge', 'just', 'just', 'local', 'company', 'provocation', 'responsibility', 'miniature', 'independent', 'independent', 'arranged', 'arranged', 'party', 'poetry', 'poetry', 'poetry', 'poetry', 'legal', 'easily', 'triangular', 'unexpected', 'unexpected', 'unexpected', 'standing', 'variable', 'father', 'loitering', 'biscuits', 'biscuits', 'biscuits', 'biscuits', 'biscuits', 'biscuits', 'available', 'separate', 'necessary', 'necessary', 'necessary', 'necessary', 'necessary', 'necessary', 'desertion', 'recent', 'receive', 'secret', 'receipt', 'remind', 'remained', 'initials', 'initials', 'initials', 'initials', 'initials', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'aint', 'ant', 'art', 'until', 'the', 'experiences', 'but', 'total', 'understand', 'understand', 'southern', 'definitely', 'definitely', 'visited', 'visited', 'visit', 'visited', 'country', 'went', 'relieve', 'forces', 'whether', 'useful', 'literature', 'valuable', 'valuable', 'delicate', 'delicate', 'delicate', 'clearing', 'splendid', 'splendid', 'opened', 'splendid', 'between', 'completely', 'account', 'century', 'secretary', 'special', 'special', 'special', 'special', 'late', 'late', 'latest', 'latest', 'perhaps', 'member', 'remember', 'remember', 'remember', 'chapter', 'chapter', 'chapter', 'can', 'various', 'february', 'parted', 'pretend', 'pretend', 'pretend', 'closing', 'rose', 'note', 'particular', 'wall', 'awful', 'fragment', 'changes', 'changes', 'laugh', 'last', 'caught', 'laugh', 'open', 'often', 'often', 'often', 'someone', 'personal', 'unique', 'practically', 'description', 'point', 'comes', 'people', 'peril', 'purple', 'decide', 'articles', 'position', 'extended', 'special', 'really', 'really', 'really', 'nothing', 'committee', 'wanted', 'wanted', 'benefit', 'deletions', 'scissors', 'scissors', 'legal', 'parallel', 'parallel', 'parallel', 'parallel', 'parallel', 'accumulation', 'composition', 'abomination', 'planet', 'fierce', 'transferred', 'mines', 'hanging', 'accepting', 'stomachs', 'stomachs', 'some', 'stomachs', 'unfortunately', 'considerable', 'access', 'singular', 'scarcely', 'scarcely', 'scarcely', 'scarcely', 'questionable', 'experience', 'experience', 'possible', 'refreshed', 'refreshed', 'represent', 'represent', 'mars', 'brass', 'visitors', 'artillery', 'decided', 'benefit', 'consider', 'failed', 'career', 'occurrence', 'occurrence', 'certain', 'some', 'view', 'establishing', 'establishing', 'different', 'ones', 'extremely', 'advisable', 'salary', 'salary', 'valley', 'valley', 'central', 'families', 'bicycle', 'bicycle', 'bicycle', 'chaise', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'curtains', 'certain', 'countess', 'curtains', 'curtains', 'martians', 'curious', 'address', 'acres', 'division', 'reason', 'management', 'inconvenience', 'incoherent', 'inconvenience', 'vacant', 'supersede', 'superseded']
print(len(pred))
print(len(truth))

a = sum(pred[i]==truth[i] for i in range(len(pred)))
print("prediction score on total test set is", a/len(pred)*100, '%')

270
270
prediction score on total test set is 56.666666666666664 %


In [64]:
score=0
total=0
for i in range(len(truth)):
    if truth[i] in tokens:
        if pred[i]==truth[i]: 
            score+=1
        total+=1
print("prediction score counting only words inside our dictionary is", score/total*100, "%")

prediction score counting only words inside our dictionary is 77.66497461928934 %


Αυτή είναι και η τιμή που λαμβάνουμε υπόψη στο score. Εφαρμόζουμε τώρα ακριβώς τα ίδια βήματα για τον unigram speller:

In [ ]:
for i in misspelled:
    inp(i, "input.fst")
    os.system("fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols input.fst inputA.fst")
    os.system("fstcompose inputA.fst let_checker.fst fourth.fst")
    os.system("fstshortestpath --nshortest=1 fourth.fst out.fst")
    os.system("fstrmepsilon out.fst out.fst")
    os.system("fsttopsort out.fst out.fst")
    os.system("fstprint out.fst word.txt")
    os.system("cut -f4 word.txt > res.txt")
    os.system("head -n -1 res.txt >> a.txt")
    os.system("echo 'new_word' >> a.txt")
    os.system("sed -i -e 's/<epsilon>//g' a.txt")


pred = []
f = open("a.txt", 'r')
lines = f.readlines()
word = ""
for l in lines:
    if l!='new_word\n':
        word+=l.rstrip()
    else:
        pred.append(word)
        word=""

print(pred)

In [67]:
pred=['consented', 'contents', 'intended', 'contents', 'beginning', 'problem', 'role', 'problem', 'people', 'siren', 'eustace', 'eustace', 'gun', 'june', 'june', 'use', 'use', 'local', 'compare', 'provocation', 'responsibility', 'miniature', 'independent', 'independent', 'arranged', 'arranged', 'party', 'poetry', 'poetry', 'piety', 'poetry', 'legal', 'scaly', 'triangular', 'unexpected', 'unexpected', 'unexpected', 'standing', 'variable', 'father', 'loitering', 'biscuits', 'disputes', 'biscuits', 'biscuits', 'biscuits', 'biscuits', 'avail', 'separate', 'necessary', 'necessary', 'necessary', 'necessary', 'necessary', 'necessary', 'desertion', 'recent', 'receive', 'resist', 'receipt', 'remind', 'remind', 'initials', 'initials', 'initials', 'initials', 'initials', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'ant', 'ant', 'art', 'india', 'her', 'experiences', 'bit', 'total', 'understand', 'understand', 'southern', 'definitely', 'definitely', 'visited', 'visited', 'visit', 'tied', 'pantry', 'men', 'relieve', 'forces', 'weather', 'useful', 'literature', 'value', 'value', 'delicate', 'delicate', 'delicate', 'clearing', 'splendid', 'splendid', 'speed', 'splendid', 'between', 'completely', 'count', 'century', 'smear', 'special', 'special', 'special', 'special', 'late', 'lets', 'latest', 'latest', 'perhaps', 'member', 'remember', 'remember', 'remember', 'chapter', 'chapter', 'chapter', 'ak', 'various', 'february', 'parted', 'pretend', 'pretend', 'pretend', 'closing', 'rate', 'woe', 'particular', 'fall', 'awful', 'armament', 'changes', 'changes', 'lash', 'la', 'laugh', 'ugh', 'open', 'often', 'often', 'often', 'someone', 'personal', 'unique', 'dramatic', 'description', 'pins', 'poses', 'people', 'peril', 'purple', 'decide', 'article', 'position', 'extended', 'special', 'real', 'rely', 'rely', 'noting', 'committee', 'wanted', 'wanted', 'benefit', 'deletions', 'scissors', 'scissors', 'leads', 'parallel', 'parallel', 'parallel', 'parallel', 'parallel', 'accumulation', 'consolation', 'abomination', 'plane', 'perch', 'transpired', 'mines', 'ringing', 'access', 'star', 'stomachs', 'some', 'stomachs', 'unfortunately', 'considerable', 'access', 'singular', 'scarcely', 'scarcely', 'scarcely', 'scarcely', 'questionable', 'experience', 'experience', 'passable', 'represent', 'refresh', 'represent', 'represent', 'ears', 'brass', 'visitors', 'allay', 'decided', 'benefit', 'consider', 'files', 'career', 'occurrence', 'occurrence', 'certain', 'ome', 'lie', 'establishing', 'establishing', 'different', 'ones', 'extremely', 'advisable', 'gale', 'allay', 'alley', 'alley', 'central', 'families', 'bicycle', 'bicycle', 'bicycle', 'chose', 'site', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'curtains', 'certain', 'countess', 'curtains', 'curtains', 'martians', 'curious', 'dress', 'acres', 'division', 'bison', 'management', 'convenient', 'invent', 'convenient', 'vacant', 'supersede', 'superseded']

a = sum(pred[i]==truth[i] for i in range(len(pred)))
print("prediction score on total test set is", a/len(pred)*100, '%')
score=0
total=0
for i in range(len(truth)):
    if truth[i] in tokens:
        if pred[i]==truth[i]: 
            score+=1
        total+=1
print("prediction score counting only words inside our dictionary is", score/total*100, "%")

prediction score on total test set is 50.74074074074074 %
prediction score counting only words inside our dictionary is 69.54314720812182 %


γ) Παρατηρούμε καταρχάς ότι τα ποσοστά δεν είναι εξαιρετικά ψηλά εξαιτίας είτε της απουσίας λέξεων στο corpus στην πρώτη περίπτωση είτε λόγω της πολύ κοντινής απόστασης κάποιων λέξεων. Συγκριτικά όμως ο word level speller έχει καλύτερο performance μια και λόγω των λίγων λέξεων του corpus τείνει προς τις πιο χρησιμοποιημένες λέξεις, γεγονός που αποδίδει φυσικά καλύτερα από το να προσανατολίζομαι προς τα πιο συχνά γράμματα.

### Βήμα 15

Αυτό που διαφοροποιείται με τις προηγούμενες περιπτώσεις είναι οι πιθανότητες μετάβασης του acceptor. Αρχικά τις υπολογίζω:

Συμπεριλαμβάνω και τις πιθανότητες μια λέξη να ξεκινάει από ένα συγκεκριμένο γράμμα:

In [15]:
import numpy as np
prob = np.zeros((len(symbols)+1,len(symbols)+1))

In [16]:
total = 0
for l in tokens:
    prev = ""
    for c in l:
        if prev!="":
            prob[symbols.index(prev)+1][symbols.index(c)+1]+=1
        else:
            prob[0][symbols.index(c)+1]+=1
        prev = c
        total+=1
print(prob[23,:])
print(total)

[0.000e+00 5.800e+01 8.400e+01 1.000e+00 0.000e+00 2.100e+01 4.200e+02
 2.000e+01 3.000e+00 3.000e+01 3.590e+02 1.760e+02 1.370e+02 1.050e+02
 7.440e+02 0.000e+00 1.000e+00 6.900e+01 0.000e+00 8.100e+01 0.000e+00
 6.690e+02 3.000e+00 1.820e+02 5.970e+02 0.000e+00 1.560e+02 1.120e+02
 2.050e+02 4.800e+01 1.900e+02 0.000e+00 1.469e+03 0.000e+00]
91044


In [17]:
for i in range(len(symbols)):
    for j in range(len(symbols)):
        prob[i][j]=prob[i][j]/total
print(prob[0,:])

[0.00000000e+00 9.45696586e-03 8.45744915e-04 5.38201309e-04
 0.00000000e+00 5.54676860e-03 1.66403058e-02 5.54676860e-03
 1.02148412e-03 6.89776372e-03 7.06251922e-03 2.69100655e-03
 2.40543034e-03 4.23970827e-03 5.37102939e-03 0.00000000e+00
 1.09837002e-04 1.99903343e-03 0.00000000e+00 4.89873028e-03
 0.00000000e+00 7.74350863e-03 0.00000000e+00 7.18333992e-03
 3.20724046e-03 0.00000000e+00 5.83234480e-03 1.25763367e-02
 8.43548175e-03 7.47989983e-03 4.28364307e-04 0.00000000e+00
 5.79939370e-03 8.00000000e+00]


Φτιάχνουμε στη συνέχεια τον acceptor:

In [18]:
import math
g = open('bigram_acceptor.fst', mode = 'w')
count=1
for word in tokens:
    prev = "<epsilon>"
    for i in range(len(word)):
        l = word[i]
        if prev!="<epsilon>":
            val = -math.log(prob[symbols.index(prev)+1][symbols.index(l)+1],10)
            g.write(str(count-1)+" "+str(count)+" "+l+" "+l +" "+str(val)+'\n')
            count+=1
        else:
            val = -math.log(prob[0][symbols.index(l)+1],10)
            g.write('0'+" "+str(count)+" "+l+" "+l+" "+str(val)+'\n')
            count+=1
        prev=l
    g.write(str(count-1)+'\n')
g.close()

In [29]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
bi_mean = np.mean(prob)
bi_mean = -math.log(bi_mean,10)

In [30]:
tot = 0
for i in range(len(symbols)):
    for j in range(len(symbols)):
        tot+=prob[i][j]
bi_mean=tot/len(symbols)/len(symbols)
bi_mean = -math.log(bi_mean,10)
print(bi_mean)

3.039438719708233


Τελικά, ούτως ώστε να λαμβάνεται υπόψη ο transducer βάζουμε σχετικά μεγάλο βάρος για το edit πχ 30. 
Έτσι κάνουμε compose κατά τα γνωστά έχοντας εφαρμόσει τα στατιστικά αποτελέσματα στον acceptor.

In [33]:

f = open('bi_mean.fst', mode = 'w')

def format_arc(src, dst, src_sym, dst_sym, w):
    f.write(str(src)+" "+str(dst)+" "+src_sym+" "+dst_sym+" "+str(w)+'\n')
    return

for i in range(len(symbols)):
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[i], w=0)
    format_arc(
        src=0, dst=0, src_sym=symbols[i], dst_sym="<epsilon>", w=30)
    format_arc(
        src=0, dst=0, src_sym="<epsilon>", dst_sym=symbols[i], w=30)
    for j in range(len(symbols)):
        if j!=i:
            format_arc(
                src=0, dst=0, src_sym=symbols[i], dst_sym=symbols[j], w=30)
f.write('0')
f.close()

In [100]:
fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols bigram_acceptor.fst bigram.fst
fstrmepsilon bigram.fst bigram.fst
fstdeterminize bigram.fst bigram.fst
fstminimize bigram.fst bigram.fst

fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols bi_mean.fst bi_trans.fst
fstarcsort bigram.fst bigram.fst
fstcompose bi_trans.fst bigram.fst bigram_checker.fst

SyntaxError: invalid syntax (<ipython-input-100-2f8b243070e0>, line 1)

Στη συνέχεια τρέχουμε το script με το test set για να δούμε αν έχει βελτιωμένα ποσοστά.

In [ ]:
for i in misspelled:
    inp(i, "input.fst")
    os.system("fstcompile --isymbols=chars.syms --osymbols=chars.syms --keep_isymbols --keep_osymbols input.fst inputA.fst")
    os.system("fstcompose inputA.fst bigram_checker.fst fourth.fst")
    os.system("fstshortestpath --nshortest=1 fourth.fst out.fst")
    os.system("fstrmepsilon out.fst out.fst")
    os.system("fsttopsort out.fst out.fst")
    os.system("fstprint out.fst word.txt")
    os.system("cut -f4 word.txt > res.txt")
    os.system("head -n -1 res.txt >> a.txt")
    os.system("echo 'new_word' >> a.txt")
    os.system("sed -i -e 's/<epsilon>//g' a.txt")


pred = []
f = open("a.txt", 'r')
lines = f.readlines()
word = ""
for l in lines:
    if l!='new_word\n':
        word+=l.rstrip()
    else:
        pred.append(word)
        word=""

print(pred)

In [37]:
pred=[]
pred=['consented', 'contents', 'intended', 'contents', 'beginning', 'problem', 'problem', 'problem', 'proper', 'siren', 'exact', 'eustace', 'gun', 'june', 'june', 'use', 'use', 'local', 'compare', 'provocation', 'responsibility', 'miniature', 'independent', 'independent', 'arranged', 'arranged', 'party', 'poetry', 'poetry', 'piety', 'poetry', 'level', 'scaly', 'triangular', 'unexpected', 'unexpected', 'unexpected', 'standing', 'variable', 'father', 'loitering', 'biscuits', 'disputes', 'biscuits', 'biscuits', 'biscuits', 'biscuits', 'avail', 'separate', 'necessary', 'necessary', 'necessary', 'necessary', 'necessary', 'necessary', 'desertion', 'recent', 'receive', 'exit', 'receipt', 'remind', 'remind', 'initials', 'initials', 'initials', 'initials', 'initials', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'magnificent', 'ant', 'ant', 'ant', 'india', 'her', 'experiences', 'bit', 'total', 'understand', 'understand', 'southend', 'definitely', 'definitely', 'visited', 'visited', 'visit', 'visited', 'pantry', 'men', 'relieve', 'forces', 'weather', 'useful', 'literature', 'valuable', 'valuable', 'delicate', 'delicate', 'delicate', 'clearing', 'splendid', 'splendid', 'spend', 'splendid', 'between', 'completely', 'count', 'century', 'sedentary', 'special', 'special', 'special', 'special', 'late', 'late', 'latest', 'latest', 'perhaps', 'member', 'remember', 'remember', 'remember', 'chapter', 'chapter', 'chapter', 'ak', 'various', 'february', 'extend', 'pretend', 'pretend', 'pretend', 'chasing', 'rate', 'woe', 'particular', 'wall', 'awful', 'armament', 'changes', 'changes', 'lash', 'la', 'laugh', 'ugh', 'open', 'often', 'often', 'often', 'someone', 'personal', 'unique', 'frantically', 'description', 'pins', 'comes', 'people', 'peril', 'purple', 'decide', 'article', 'position', 'extended', 'special', 'real', 'rely', 'rely', 'noting', 'committee', 'wanted', 'wanted', 'benefit', 'deletions', 'scissors', 'scissors', 'level', 'parallel', 'parallel', 'parallel', 'parallel', 'parallel', 'accumulation', 'consolation', 'abomination', 'plane', 'perch', 'transfixed', 'mines', 'ringing', 'accepting', 'star', 'stomachs', 'some', 'stomachs', 'unfortunately', 'considerable', 'access', 'singular', 'scarcely', 'scarcely', 'scarcely', 'scarcely', 'questionable', 'experience', 'experience', 'passable', 'represent', 'refresh', 'represent', 'represent', 'ears', 'brass', 'visitors', 'artillery', 'decided', 'benefit', 'consider', 'files', 'career', 'occurrence', 'occurrence', 'certain', 'ome', 'lie', 'establishing', 'establishing', 'different', 'ones', 'extremely', 'advisable', 'gale', 'allay', 'alley', 'alley', 'central', 'families', 'bicycle', 'bicycle', 'bicycle', 'chose', 'site', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'opposite', 'curtains', 'certain', 'countess', 'curtains', 'curtains', 'curtains', 'curious', 'dress', 'acres', 'division', 'bison', 'management', 'convenient', 'incoherent', 'convenient', 'vacant', 'supersede', 'superseded']


a = sum(pred[i]==truth[i] for i in range(len(pred)))
print("prediction score on total test set is", a/len(pred)*100, '%')
score=0
total=0
for i in range(len(truth)):
    if truth[i] in tokens:
        if pred[i]==truth[i]: 
            score+=1
        total+=1
print("prediction score counting only words inside our dictionary is", score/total*100, "%")

prediction score on total test set is 52.59259259259259 %
prediction score counting only words inside our dictionary is 72.08121827411168 %


Παρατηρούμε ότι έχει ελαφρώς καλύτερη απόδοση από τον unigram speller αφού προσφέρει περισσότερη πληροφορία στο μοντέλο, αλλά απέχει περίπου 5% από την ακρίβεια του μοντέλου που λαμβάνει υπόψη τη συχνότητα εμφάνισης των λέξεων στο λεξικό.

# Μέρος 2: Χρήση σημασιολογικών αναπαραστάσεων για ανάλυση συναισθήματος

### Βήμα 16

Χρησιμοποιώντας τον κώδικα που μας δίνεται διαβάζουμε τα δεδομένα που κατεβάσαμε.

In [49]:
import os

data_dir = './aclImdb/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
pos_train_dir = os.path.join(train_dir, 'pos')
neg_train_dir = os.path.join(train_dir, 'neg')
pos_test_dir = os.path.join(test_dir, 'pos')
neg_test_dir = os.path.join(test_dir, 'neg')

# For memory limitations. These parameters fit in 8GB of RAM.
# If you have 16G of RAM you can experiment with the full dataset / W2V
MAX_NUM_SAMPLES = 5000
# Load first 1M word embeddings. This works because GoogleNews are roughly
# sorted from most frequent to least frequent.
# It may yield much worse results for other embeddings corpora
NUM_W2V_TO_LOAD = 1000000

import numpy as np

SEED = 42

# Fix numpy random seed for reproducibility
np.random.seed(42)

try:
    import glob2 as glob
except ImportError:
    import glob

import re

def strip_punctuation(s):
    return re.sub(r'[^a-zA-Z\s]', ' ', s)

def preprocess(s):
    return re.sub('\s+',' ', strip_punctuation(s).lower())

def tokenize(s):
    return s.split(' ')

def preproc_tok(s):
    return tokenize(preprocess(s))

def read_samples(folder, preprocess=lambda x: x):
    samples = glob.iglob(os.path.join(folder, '*.txt'))
    data = []
    for i, sample in enumerate(samples):
        if MAX_NUM_SAMPLES > 0 and i == MAX_NUM_SAMPLES:
            break
        with open(sample, 'r', encoding="utf8") as fd:
            x = [preprocess(l) for l in fd][0]
            data.append(x)
    return data

def create_corpus(pos, neg):
    corpus = np.array(pos + neg)
    y = np.array([1 for _ in pos] + [0 for _ in neg])
    indices = np.arange(y.shape[0])
    np.random.shuffle(indices)
    return list(corpus[indices]), list(y[indices])

In [50]:
ntrain=read_samples(neg_train_dir, preproc_tok)
ntest=read_samples(neg_test_dir, preproc_tok)
ptrain=read_samples(pos_train_dir,preproc_tok)
ptest=read_samples(pos_test_dir,preproc_tok)

### Βήμα 17

α) Η μετρική tf-idf μας δείχνει πόσο σημαντική είναι μία λέξη για ένα corpus. Ουσιαστικά, όσο μεγαλύτερη συχνότητα εμφάνισης έχει μια λέξη στο corpus, τόσο μεγαλύτερο βάρος θα έχει στο bag of words.

β) Μετράμε ουσιαστικά πόσες φορές υπάρχει μια λέξη του λεξιλογίου σε ένα σχόλιο δημιουργώντας ένα vector για κάθε δείγμα. Αυτό το vector θα έχει μέγεθος ίσο με το μέγεθος του λεξιλογίου. Έτσι αρχικά χωρίζουμε τα σχόλια σε ntrain (αρνητικά του train), ptrain (θετικά του train), ntest (αρνητικά του test), ptest (θετικά του test). Τα φέρνουμε σε μορφή τέτοια ώστε να αποτελούν μια λίστα από strings και τα δίνουμε όλα στον vectorizer.

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
ntrain = [str(sublist) for sublist in ntrain]

ptrain = [str(sublist) for sublist in ptrain]

ptest = [str(sublist) for sublist in ptest]

ntest = [str(sublist) for sublist in ntest]

total = ntrain + ptrain + ntest + ptest
X = vectorizer.fit_transform(total)
print(len(X.toarray()[0]))

65876


Η έξοδος είναι μια λίστα από vectors, όσα είναι και τα δείγματά μας. Μετατρέπουμε τον πίνακα σε np.array().

In [17]:
X = X.toarray()
print(len(X), len(X[0]))
# καθε set έχει 5000 samples

20000 65876


In [18]:
X = np.array(X)

Χωρίζουμε τώρα τα δεδομένα σε train και test. Στα μισά δεδομένα κάθε set βάζουμε label 0 που σημαίνει ότι είναι αρνητικά σχόλια και στα υπόλοιπα μισά βάζουμε ότι είναι θετικά με label 1.

In [19]:
X_train = X[0:10000]
X_test = X[10000:]
y_train = [0 if i<5000 else 1 for i in range(10000)]
y_train = np.array(y_train)
y_test = y_train

In [20]:
print(X_train.shape)
print(y_train.shape)
print(y_train)

(10000, 65876)
(10000,)
[0 0 0 ... 1 1 1]


γ) Εκπαιδεύουμε στη συνέχεια τον προβλέπτη Logistic Regression δίνοντας του ως είσοδο τα train δεδομένα και υπολογίζουμε το score του με τα test δεδομένα.

In [21]:
from sklearn.linear_model import LogisticRegression

In [23]:
reg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)

C:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [24]:
print(reg.score(X_test, y_test))

0.8564


δ) Με τον ίδιο τρόπο αντί για CountVectorizer χρησιμοποιούμε τον TfidfVectorizer και επαναλαμβάνουμε τα ίδια βήματα για να βρούμε το score.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(total)
X = X.toarray()
X_train = X[0:10000]
X_test = X[10000:]
y_train = [0 if i<5000 else 1 for i in range(10000)]
y_train = np.array(y_train)
y_test = y_train
reg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
print(reg.score(X_test, y_test))

0.8757


Παρατηρούμε ότι πράγματι υπήρξε μια μικρή βελτίωση της τάξης του 2% για τον Tfidf Vectorizer, μια και αυτή η αναπαράσταση μας δίνει με μεγαλύτερη ακρίβεια το πόσο σημαντική είναι μια λέξη για την αξιολόγηση ενός σχολίου.

### Βήμα 18
α) Έχοντας το λεξιλόγιο και όλες τις λέξεις του corpus βρίσκω το αντίστοιχο ποσοστό OOV words.

In [27]:
flat = []
for sublist in senten:
    for item in sublist:
        flat.append(item)

In [29]:
total1=0
for i in flat:
    if i not in voc:
        total1+=1
print("percentage of words not in the vocabulary is", total/len(flat)*100, "%")

8.727217283197673 %


β) Παίρνοτας την matrix μορφή των embeddings που κατασκευάσαμε στο ερώτημα 9, φτιάχνουμε για κάθε σχόλιο το αντίστοιχο vector παίρνοντας τον μέσο όρο των διανυσμάτων w2vec κάθε λέξης του σχολίου.
Αν μια λέξη δεν υπάρχει στο vocabulary βάζουμε όλα μηδενικά.

In [33]:
X1 = to_embeddings_Matrix(model)

In [70]:
len(total)

20000

In [58]:
def tokenize(s):
    s = s.strip()
    s = s.lower()
    news=""
    for a in s:
        if a==' ' or a=='\n': news+=' '
        elif a.isalpha(): news+=a
    return news.split()

In [73]:
all_emb = np.zeros(shape=(len(total), len(X1[0][0,:])))
count = 0
for i in total:
    emb = np.zeros(len(X1[0][0,:]))
    ws = tokenize(i)
    for j in ws:
        if j in X1[1]:
            emb += X1[0][X1[2][j],:]
        else:
            emb += np.zeros(len(X1[0][0,:]))
    emb /= len(j)
    all_emb[count,:] = emb
    count+=1

Έχοντας τα δεδομένα σε κατάλληλη μορφή τα δίνουμε ως είσοδο στον Logistic Regression

In [77]:
reg1 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(all_emb[0:10000], y_train)
print(reg1.score(all_emb[10000:], y_test))

0.7046


C:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Το μοντέλο μας πετυχαίνει score 70% το οποίο είναι συγκριτικά αρκετά μικρότερο σε σχέση με τα προηγούμενα ερωτήματα. Αυτό οφείλεται στο γεγονός ότι βασιστήκαμε σε προεκπαιδευμένα embeddings τα οποία δεν είχαν μεγάλη σχέση με τα training reviews μια και πιθανόν να υπήρχε αρκετά διαφορετική χρήση λέξεων και προφανώς λέξεις που δεν περιλαμβάνονταν καν στο λεξικό.

Ως αποτέλεσμα, δεν έγινε τόσο καλή αξιολόγηση των reviews και καταλήξαμε σε αυτό το 70%

γ)δ)

In [89]:
from gensim import models

model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',
binary=True, limit=NUM_W2V_TO_LOAD)


model

In [91]:
model.most_similar(['king'])

[('kings', 0.7138045430183411),
 ('queen', 0.6510956883430481),
 ('monarch', 0.6413194537162781),
 ('crown_prince', 0.6204219460487366),
 ('prince', 0.6159993410110474),
 ('sultan', 0.5864823460578918),
 ('ruler', 0.5797567367553711),
 ('princes', 0.5646552443504333),
 ('throne', 0.5422105193138123),
 ('royal', 0.5239794254302979)]

In [95]:
import random
test = ['clue']
for i in range(10):
    test.append(voc[random.choice(range(len(voc)))])

for i in test:
    print(i)
    print(model.most_similar(i))
    print()

clue
[('clues', 0.6851588487625122), ('inkling', 0.6116672158241272), ('slightest_clue', 0.5809398889541626), ('know', 0.5430039763450623), ('foggiest_idea', 0.5362423658370972), ('hint', 0.5173760652542114), ('indication', 0.5164186954498291), ('faintest_idea', 0.511157751083374), ('wonder', 0.5002449154853821), ('hints', 0.492409884929657)]

success
[('successes', 0.724018394947052), ('sucess', 0.7154811024665833), ('successful', 0.6167577505111694), ('Success', 0.6160483956336975), ('succes', 0.6056337356567383), ('resounding_success', 0.5794840455055237), ('succcess', 0.5783605575561523), ('successs', 0.5095175504684448), ('hugely_successful', 0.4963102638721466), ('accomplishment', 0.49159348011016846)]

whenever
[('wherever', 0.7060423493385315), ('Whenever', 0.6958863735198975), ('everytime', 0.6308388710021973), ('anytime', 0.6082679033279419), ('whatever', 0.5835779905319214), ('frequently', 0.5679360628128052), ('invariably', 0.5635004043579102), ('if', 0.5556199550628662), (

Επειδή το δείγμα μας είναι πολύ μεγαλύτερο, έχει γίνει πολύ καλή εκπαίδευση του μοντέλου με βάση τα embeddings. Έτσι είναι φανερό ότι τα αποτελέσματα είναι άμεσα συσχετισμένα με τις λέξεις που δίνονται ως είσοδος. Για παράδειγμα στη λέξη 'son' ως αποτέλεσμα δίνονται και άλλες μορφές συγγένειας, ενώ στο χρώμα 'color' 'εχουμε ως έξοδο άλλα χρώματα.

ε) Όπως έβγαλα αναπαραστάσεις με βάση το αρχικό corpus βγάζω τώρα με το google reviews που κατέβασα και κάνω πάλι αξιολόγηση του συστήματος με Logistic Regression.

In [99]:
def to_embeddings_Matrix(model):  
    embedding_matrix = np.zeros((len(model.vocab), model.vector_size))
    word2idx = {}
    for i in range(len(model.vocab)):
        embedding_matrix[i] = model[model.index2word[i]]
        word2idx[model.index2word[i]] = i
    return embedding_matrix, model.index2word, word2idx

X2 = to_embeddings_Matrix(model)

In [116]:
X2[2].get('whynot')

In [157]:
import datetime
start = datetime.datetime.now()
google_emb = np.zeros(shape=(len(total), len(X2[0][0,:])))
count = 0
for i in total:
    emb = np.zeros(len(X2[0][0,:]))
    ws = tokenize(i)
    for j in ws:
        if X2[2].get(j) is not None:
            emb += X2[0][X2[2][j],:]
        else:
            emb += np.zeros(len(X2[0][0,:]))
    emb /= len(i)
    google_emb[count,:] = emb
    count+=1
end = datetime.datetime.now()
total_time = end - start
print(total_time)

0:00:30.087458


In [158]:
reg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(google_emb[0:10000], y_train)
print(reg.score(google_emb[10000:], y_test))

0.7394


Φυσικά πολύ καλύτερο ποσοστό από το 70% στο οποίο είχαμε εκπαιδεύσει το σύστημά μας με άλλο λεξιλόγιο.

στ) Τώρα κάνουμε το ίδιο αλλά πολλαπλασιάζουμε κάθε γραμμή με το βάρος tf-idf

In [159]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(total)
names = vectorizer.get_feature_names()

In [160]:
 names[5]

'aaaaargh'

In [163]:
import datetime
start = datetime.datetime.now()
google_idf_emb = np.zeros(shape=(len(total), len(X2[0][0,:])))
count = 0
for i in total:
    emb = np.zeros(len(X2[0][0,:]))
    ws = tokenize(i)
    mysum=0
    for j in i:
        if X2[2].get(j) is not None and voc.get(j) is not None:
            emb += X[count,voc[j]]*X2[0][X2[2][j],:]
            mysum+=X[count,voc[j]]
        else:
            emb += np.zeros(len(X2[0][0,:]))
    if mysum!=0:
        emb /= mysum
    google_idf_emb[count,:] = emb
    count+=1
end = datetime.datetime.now()
total_time = end - start
print(total_time)

0:02:31.159473


Τρέχω τώρα τον classifier με βάση τα παραπάνω δεδομένα:

In [164]:
reg2 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(google_idf_emb[0:10000], y_train)
print(reg2.score(google_idf_emb[10000:], y_test))

0.5


### Βήμα 19

α) Δοκιμάζουμε τους ταξινομητές knn, svm με είσοδο τα google embeddings.

In [165]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(google_emb[0:10000], y_train) 
print(neigh.score(google_emb[10000:], y_test))

0.7313


In [166]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(google_emb[0:10000], y_train) 
print(clf.score(google_emb[10000:], y_test))

0.7002


In [168]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(google_emb[0:10000], y_train) 
print(clf.score(google_emb[10000:], y_test))

0.6867


Παρατηρούμε ότι ο Nearest Neighbours απέδωσε εξίσου καλά με τον Logistic Regression. Ο SVM αντίστοιχα είχε μειωμένα ποσοστά τα οποία διαφοροποιούνται ανάλογα την επιλογή του kernel.

β) Παρόμοια με το word2vec δοκιμάζουμε το FastText το οποίο μπορούμε να το δηλώσουμε εύκολα ως embedding από τις έτοιμες βιβλιοθήκες του gensim.

In [169]:
from gensim.models import FastText
model_ted = FastText(senten, size=100, window=5, min_count=5, workers=4,sg=1)

In [170]:
model_ted.train(senten, total_examples=len(senten), epochs=1000)

In [176]:
voc = model_ted.wv.index2word

import random
test = ['clue']
for i in range(10):
    test.append(voc[random.choice(range(len(voc)))])

for i in test:
    print(i)
    print(model_ted.most_similar(i))
    print()

clue
[('hoped', 0.4695552885532379), ('excellent', 0.45561397075653076), ('trademark', 0.4519026577472687), ('invariably', 0.44242244958877563), ('race', 0.4399247169494629), ('trace', 0.4272076487541199), ('cheetah', 0.42570364475250244), ('rule', 0.41493621468544006), ('leadenhall', 0.4120497703552246), ('missed', 0.40796148777008057)]

drawingroom
[('watched', 0.469998836517334), ('sittingroom', 0.4655047655105591), ('receive', 0.46206986904144287), ('paddington', 0.45110204815864563), ('play', 0.44920432567596436), ('fragments', 0.44754114747047424), ('choose', 0.4471031427383423), ('status', 0.44553062319755554), ('daring', 0.445462167263031), ('dropping', 0.44527512788772583)]

medium
[('physical', 0.6026408076286316), ('providing', 0.5887846946716309), ('format', 0.5840810537338257), ('copies', 0.5456855297088623), ('works', 0.5391372442245483), ('electronic', 0.5306658744812012), ('possessed', 0.5300577282905579), ('liability', 0.5234711170196533), ('derivative', 0.519077539443

C:\anaconda\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


Σημασιολογικά, με μια πρώτη ματιά τα similarities είναι όσο επιτυχημένα είναι και στο word2vec.

Αυτό που αξίζει να σημειώσουμε είναι η εμφάνιση λέξεων με ίδια ρίζα, όπως στα ζεύγη "distributed"-"distribute" ή "aggreed"-"aggreement". Αυτό συμβαίνει γιατί στο FastText οι λέξεις παρουσιάζονται ως ένα άθροισμα διανυσμάτων, δηλαδή n-grams. Έτσι εμφανίζονται και λέξεις με ορθογραφική και ετοιμολογική σχέση και όχι μόνο σημασιολογική.

Αυτό το γεγονός φαίνεται στα παραπάνω από αποτελέσματα τύπου "drawingroom" -"sittingroom".